In [30]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime

In [27]:
%run ./keys.ipynb
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [175]:
df = pd.read_csv('../../../../sarahr/Ironhack/political_spanish_sentiment/political_spanish_sentiment_scripts/political_spanish_sentiment_ddbb.csv')

In [149]:
def extracting_mentions(query):
    """
    This function extract a data frame with the result of a query.
    """
    
    # date and amount information
    count = 10

    today = datetime.date.today()
    yest = today - datetime.timedelta(days=1)
    yest2 = today - datetime.timedelta(days=7)
    tomorrow = today + datetime.timedelta(days=1)
    since=yest2
    until=today
    
    print(f'Until: {until}')
    print(f'Since: {since}')
    
    # getting the mentions
    
    print(f'Getting mentions for {query} since {since} until {until}.')
    
    mentions = [tweet for tweet in tweepy.Cursor(api.search, 
                                                q=query + ' -filter:retweets', 
                                                lang="es", 
                                                tweet_mode='extended',
                                                 until=until,
                                                 since=since,
                                                result_type="recent").items(count)]
    mentions_json = [tweet._json for tweet in mentions]
    df_mentions = pd.json_normalize(mentions_json)
    
    # selecting useful columns
    columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count']
    
    df_mentions_filtered = df_mentions[columns_selected]
    
    # cleaning date time
    df_mentions_filtered.loc[:, 'created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
    
    # cleaning source of the tweet
    list_sources = list(df_mentions_filtered['source'])
    df_mentions_filtered.loc[:, 'source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]
    
    return df_mentions_filtered

In [162]:
def extracting_tweets(df, account, string_party, string_post_type, string_campaign):
    """
    This function extract a data frame with the tweets of an account
    and filter the data frame with the columns selected.
    Input:
    - df: bbdd
    - account: twitter account.
    - string_party: 'Partido Popular', 'PSOE'...
    - string_post_type: 'publicación'
    - string_campaing: 'general' or 'madrid'
    Output: a data frame with the last tweets of the accoount indicated.
    """
    
    # parameters
    last_tweet_id = max(df['id'])
    count = 10
    
    # extracting the tweets
    tweets = [tweet for tweet in tweepy.Cursor(api.user_timeline,
                                           screen_name=account,
                                           tweet_mode='extended',
                                           exclude_replies=True,
                                               max_id=last_tweet_id,
                                              include_rts=False).items(count)]
    # creating the data frame
    tweets_json = [tweet._json for tweet in tweets]
    df_tweets = pd.json_normalize(tweets_json)
    
    # selecting useful columns
    columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count']
    
    df_tweets_filtered = df_tweets[columns_selected]
    
    # cleaning date time
    df_tweets_filtered.loc[:, 'created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
    
    # cleaning source of the tweet
    list_sources = list(df_tweets_filtered['source'])
    df_tweets_filtered.loc[:, 'source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]
    
    # adding a column with the party
    df_tweets_filtered['partido'] = string_party
    
    # adding a column with the type of post (publicación o mención)
    df_tweets_filtered['tipo de post'] = string_post_type
    
    # adding a column with the type of post (publicación o mención)
    df_tweets_filtered['campaña'] = string_campaign
    
    # extract hashtags (column ['full_text'])
    list_hashtags = list(df_tweets_filtered['full_text'])
    hashtags = [re.findall(r"#(\w+)", tweet) for tweet in list_hashtags]
    df_tweets_filtered['hashtags'] = hashtags
    
    return df_tweets_filtered

In [164]:
extracting_tweets(df, 'populares', 'Partido Popular', 'publicación', 'general')

<ipython-input-162-6d9ccb3e0623>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['partido'] = string_party
<ipython-input-162-6d9ccb3e0623>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['tipo de post'] = string_post_type
<ipython-input-162-6d9ccb3e0623>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,campaña,hashtags
0,Partido Popular,2021-04-28 15:27:24+00:00,1387428240600485894,"🇪🇺 @DolorsMM: ""Para ganar la batalla contra la...","[0, 213]",Twitter Web App,37,69,822275,5291,102873,Partido Popular,publicación,general,[]
1,Partido Popular,2021-04-28 13:23:53+00:00,1387397158920495104,"""Es evidente que este Gobierno que lleva alime...","[0, 255]",Twitter Media Studio - LiveCut,122,322,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
2,Partido Popular,2021-04-28 13:18:49+00:00,1387395882941812740,"""Vamos a reducir los precios públicos de los g...","[0, 231]",Twitter Media Studio - LiveCut,57,148,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
3,Partido Popular,2021-04-28 13:14:48+00:00,1387394873205133318,"""La Comunidad es Madrid es la segunda de toda ...","[0, 252]",Twitter Media Studio - LiveCut,47,93,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
4,Partido Popular,2021-04-28 13:12:56+00:00,1387394401022922756,"""Hemos construido dos hospitales públicos en t...","[0, 268]",Twitter Media Studio - LiveCut,143,339,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
5,Partido Popular,2021-04-28 13:09:08+00:00,1387393447078899712,"""Las elecciones del próximo 4 de mayo pueden s...","[0, 218]",Twitter Media Studio - LiveCut,55,100,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
6,Partido Popular,2021-04-28 13:05:05+00:00,1387392426076262403,"""Son muchos los socialdemócratas huérfanos que...","[0, 165]",Twitter Media Studio - LiveCut,64,117,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
7,Partido Popular,2021-04-28 13:03:22+00:00,1387391995501498370,"""Durante esta crisis ha quedado claro que la ú...","[0, 276]",Twitter Media Studio - LiveCut,48,83,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
8,Partido Popular,2021-04-28 12:58:28+00:00,1387390760857841665,"""El 4 de mayo son unas elecciones trascendenta...","[0, 266]",Twitter Media Studio - LiveCut,49,91,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"
9,Partido Popular,2021-04-28 12:38:32+00:00,1387385744147292161,📡 @IdiazAyuso participa en un almuerzo coloqui...,"[0, 117]",Twitter Media Studio,38,91,822275,5291,102873,Partido Popular,publicación,general,"[YoConAyuso, VotaLIBERTAD]"


In [171]:
df[df['user.name'] == 'Partido Popular']['id'].max()

1387428240600485894

In [161]:
for ids in df['id']:
    if df['user.name'] == 'Partido Popular':
        print(ids.max())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [74]:
list_names = list(df['user.name'])

In [75]:
df.loc[:,'user.name'] = [re.findall(r'(?i)\b[a-z]+\b', s) for s in list_names]

/Users/sarahr/miniconda3/envs/ironhack_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [86]:
df.loc[:,'user.name'] = df['user.name'].apply(lambda x: re.findall(r'(?i)\b[a-z]+\b', x))

In [88]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [91]:
df.loc[:,'user.name'] = df['user.name'].apply(remove_emoji)

In [181]:
# machine learning
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [185]:
def sentiment_classificaton(df, column):
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

    list_tweets = column.tolist()

    results = classifier(list_tweets[:8000])

    sentiment_list = []
    scores = []

    for result in results:
        scores_list = result['score']
        scores.append(scores_list)

        for i in range(len(result)):
            results_list = result['label'][i]
            if results_list != " ":
                sentiment_list.append(results_list)

    df['sentimiento'] = sentiment_list
    df['scores'] = scores

    return df

In [182]:
sentiment_classificaton(df, df['full_text'][:10])

ValueError: Length of values (10) does not match length of index (202571)

In [183]:
df[:10]

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags,campaña
0,0,Sarabia De Boj,2021-04-28 13:35:34+00:00,1387400098469064706,@FreedomWarro @Valentin_GarAl @asancfisac @Idi...,"[173, 215]",['Twitter for Android'],0,0,7,259,16,Partido Popular,mención,[],general
1,1,jesu lopez lo,2021-04-28 13:35:20+00:00,1387400036624080899,Toni Cantó ese filósofo que estableció el teo...,"[0, 215]",['Twitter for iPhone'],0,0,273,272,4480,Partido Popular,mención,[],general
2,2,Miguel Benito,2021-04-28 13:35:14+00:00,1387400014406852611,"@populares @IdiazAyuso ¿""El mejor hospital de ...","[23, 281]",['Twitter Web App'],0,0,145,225,23866,Partido Popular,mención,[],general
3,3,Bernardo Diaz Martínez,2021-04-28 13:35:08+00:00,1387399988846776322,@publico_es No están ellos sacando todos los d...,"[12, 192]",['Twitter Web App'],0,0,0,87,310,Partido Popular,mención,[],general
4,4,Ritxi ·🔻·,2021-04-28 13:35:03+00:00,1387399968097509377,"@AngelBlancoMo @populares Bueno, yo hablaba de...","[26, 304]",['Twitter for Android'],0,0,63,346,1130,Partido Popular,mención,[],general
5,5,Mentxu,2021-04-28 13:34:55+00:00,1387399934052347915,Murcia. ¿Se pueden dejar las decisiones a la a...,"[0, 280]",['Twitter for Android'],0,0,520,878,13901,Partido Popular,mención,[],general
6,6,Paula 🍍,2021-04-28 13:34:46+00:00,1387399896186171397,"@populares se llama sentido del humor, ironía ...","[11, 78]",['Twitter Web App'],0,0,432,294,62651,Partido Popular,mención,[],general
7,7,Common_sense2,2021-04-28 13:34:37+00:00,1387399856755560452,@BieitoConde @populares Comprensión lectora de...,"[24, 137]",['Twitter for Android'],0,0,194,427,1306,Partido Popular,mención,[],general
8,8,ana fernaud,2021-04-28 13:34:34+00:00,1387399846332669952,El PP se ha hartado de llamarle colaborador de...,"[0, 128]",['Twitter for Android'],0,0,191,319,31579,Partido Popular,mención,[],general
9,9,Bea Fanjul,2021-04-28 13:34:09+00:00,1387399740145422340,@torosleganes @IdiazAyuso @ComunidadMadrid @no...,"[86, 103]",['Twitter for iPhone'],0,0,105845,1697,8732,Partido Popular,mención,[],general
